# Prediction code for Utah FORGE data

In [1]:
import os
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import optimizers, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import *
from keras.models import load_model
from keras.utils import Sequence
import tensorflow.keras.backend as K
from numpy.random import seed

from utils import *
from model import *

# Define paths for figures and models
out_path = os.getenv('HOME')+"/Yang/Data/DAS_data/German/raw_data/GrSk4_VP09_processed/"
fig_path = os.getenv('HOME')+"/Yang/Data/DAS_data/German/raw_data/GrSk4_VP09_processed/figs/"

# Create directories if they do not exist
os.makedirs(out_path, exist_ok=True)
os.makedirs(fig_path, exist_ok=True)

2025-03-06 15:45:24.903190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-06 15:45:24.903215: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-06 15:45:24.904200: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 15:45:24.908879: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-06 15:45:25.424212: W tensorflow/compiler/tf2

In [2]:
# Paths and parameters
# Patch parameters
w1, w2 = 24, 24
z1, z2 = 6, 6
batch_size = 1024
drop_rate = 0.2
epochs = 50
v = 0.05
data_path = os.getenv('HOME')+'/Yang/Data/DAS_data/German/raw_data/GrSk4_VP09_raw/'
## load trained model
model = load_model('./model/trained_model_1.h5')


for i in range(1, 16):
    data = np.load(f'{data_path}data_{i}.npy')
    print(f'current processed data {i}')

    # Apply patching
    data_noisy = yc_patch(data, w1, w2, z1, z2)

    ## prediction
    out = model.predict(data_noisy)
    out = np.transpose(out)

    # unpatching
    [n1, n2] = data.shape
    predicted = yc_patch_inv(out,n1,n2,w1,w2,z1,z2)

    np.save(f'{out_path}processed_data_{i+1}.npy', predicted)

    # Constants
    xx, yy = 100, 0.04
    v1 = 1
    
    # Data dimensions and time axis
    num_samples = data.shape[0]
    num_samples_x = data.shape[1]
    time_interval = 0.001
    time = np.arange(0, num_samples) * time_interval
    
    # Annotation positions
    mm = -0.03
    nn = 1.05
    
    # Create the figure and subplots
    fig = plt.figure(figsize=(10, 5))
    
    # Plot 1: Raw data
    ax1 = fig.add_subplot(1, 3, 1)
    im1 = ax1.imshow(
        data, cmap=cseis(), vmin=-v, vmax=v, aspect='auto', extent=(1, num_samples_x, time[-1], 0)
    )
    ax1.set_xlabel("Trace", fontsize=12)
    ax1.set_ylabel("Time (s)", fontsize=12)
    ax1.set_title('Raw Data', fontsize=14)
    ax1.annotate('(a)', xy=(mm, nn), xycoords='axes fraction', fontsize=14, fontweight='bold', va='top')
    
    # Plot 2: Denoised data
    ax2 = fig.add_subplot(1, 3, 2)
    im2 = ax2.imshow(
        predicted, cmap=cseis(), vmin=-v, vmax=v, aspect='auto', extent=(1, num_samples_x, time[-1], 0)
    )
    ax2.set_xlabel("Trace", fontsize=12)
    ax2.set_yticks([])  # Remove y-ticks for cleaner appearance
    ax2.set_title('Denoised Data', fontsize=14)
    ax2.annotate('(b)', xy=(mm, nn), xycoords='axes fraction', fontsize=14, fontweight='bold', va='top')
    
    # Plot 3: Removed noise
    ax3 = fig.add_subplot(1, 3, 3)
    im3 = ax3.imshow(
        (data - predicted), cmap=cseis(), vmin=-v, vmax=v, aspect='auto', extent=(1, num_samples_x, time[-1], 0)
    )
    ax3.set_xlabel("Trace", fontsize=12)
    ax3.set_yticks([])  # Remove y-ticks for cleaner appearance
    ax3.set_title('Removed Noise', fontsize=14)
    ax3.annotate('(c)', xy=(mm, nn), xycoords='axes fraction', fontsize=14, fontweight='bold', va='top')
    
    # Adjust layout and add colorbar
    plt.tight_layout()
    fig.subplots_adjust(right=0.9)  # Make space for colorbar
    
    # Add colorbar
    cbar_ax = fig.add_axes([0.905, 0.2, 0.015, 0.6])  # [left, bottom, width, height]
    cb = plt.colorbar(im3, cax=cbar_ax)
    cb.ax.tick_params(labelsize=8)
    cbar_ticks = np.linspace(-v, v, 5)  # 5 ticks evenly spaced
    cb.set_ticks(cbar_ticks)
    cb.set_ticklabels([f'{tick:.2f}' for tick in cbar_ticks])
    
    # Save and show the plot
    output_file = f'{fig_path}denoised_data_{i+1}.png'
    plt.savefig(output_file, dpi=200)
    plt.show()

2025-03-06 15:45:27.513588: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-06 15:45:27.513787: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-06 15:45:27.546142: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required l

current processed data 1
1180/1180 [==============================] - 3s 2ms/step
current processed data 2
1180/1180 [==============================] - 2s 2ms/step
current processed data 3
1180/1180 [==============================] - 2s 2ms/step
current processed data 4
1180/1180 [==============================] - 2s 2ms/step
current processed data 5
1180/1180 [==============================] - 2s 2ms/step
current processed data 6
1180/1180 [==============================] - 2s 2ms/step
current processed data 7
1180/1180 [==============================] - 2s 2ms/step
current processed data 8
1180/1180 [==============================] - 2s 2ms/step
current processed data 9
1180/1180 [==============================] - 2s 2ms/step
current processed data 10
1180/1180 [==============================] - 2s 2ms/step
current processed data 11
1180/1180 [==============================] - 2s 2ms/step
current processed data 12
1180/1180 [==============================] - 3s 2ms/step
current proce